# Functionality: visualization of functional connectivity

<ul>
    <li>visualize the ciCOH in mild and normal state</li>
</ul>

## Import Packages 

In [1]:
import os, sys
import numpy as np
import pandas as pd

import pickle

from igraph.drawing.text import TextDrawer
import cairo

In [2]:
# import_nbmodule used for import package in .ipynb
import import_nbmodule

# extract the exp folder path
currfolder = os.getcwd()
codefolder = currfolder[0 : currfolder.find('code')+len('code')]

# add path the exp folder
sys.path.append(codefolder)


# import animal_folder function in util/folder_extract.pynb 
from util.folder_extract import exp_subfolders, code_corresfolder

# import graphVisual in connAnalyTool/graphVisual.pynb 
from connAnalyTool import graphVisual

importing Jupyter notebook from /home/lingling/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/code/util/folder_extract.ipynb
importing Jupyter notebook from /home/lingling/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/code/connAnalyTool/graphVisual.ipynb


## exp subfolders & code_corresfolder

In [3]:
_, _, pipelinefolder, _ = exp_subfolders()

In [4]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [5]:
nb_name = nb_name[0: nb_name.find('.ipynb')]

# corresfolder
corresfolder = code_corresfolder(os.getcwd(), nb_name)

## global parameters

In [6]:
# threshold lowweight
lowweight = 0.25

# frequency
freq = [9, 11]
strfreq = 'freq' + str(freq[0]) + '_' + str(freq[1])


# eventphase
eventphase = 'reach'

# area to show
helmes_show = "left_right" # left_right, left or right

In [30]:
### text setup for brain areas ###
texts_area = dict()

# position of area [x, y, fontsize]

texts_area['Cd'] = [10, 30, 20]

texts_area['VPLo'] = [10, 100, 20]

texts_area['VLo'] = [10, 155, 20]

texts_area['VA'] = [10, 225, 20]

texts_area['MC'] = [10, 275, 20]

texts_area['SMA'] = [10, 345, 20]

texts_area['M1'] = [10, 510, 20]


texts_area['STN C5-6'] = [280, 150, 20]
texts_area['GP C6-7'] = [375, 265, 20]

pos_title = [275, 55, 25]

## input setup

In [8]:
inputfolder_ciCOH = os.path.join(pipelinefolder, 'NHP_Pinky','1_dataAnaly', 'FCAnaly', 'SKT', 'm1_STKData_calcciCOH_reachPhase_Fre9-11Hz')
filename_ciCOH = 'ciCOH_STK_freq9_11reach.pickle'

ciCOHfile = os.path.join(inputfolder_ciCOH, filename_ciCOH)

In [9]:
chnInf_folder = os.path.join(pipelinefolder, 'NHP_Pinky','1_dataAnaly', 'FCAnaly', 'SKT', 'm1_simulatedLocs_M1ThaSMADBS')

chnInf_file = os.path.join(chnInf_folder, 'chn_brainArea_simCoord_M1ThaSMADBS.csv')

## savefile setup

In [10]:
if helmes_show == "left_right":
    tag_showleft, tag_showright = True, True
elif helmes_show == "left":
    tag_showleft, tag_showright = True, False
elif helmes_show == "right":
    tag_showleft, tag_showright = False, True

In [11]:
title_pre = eventphase + ":"

In [12]:
savefolder = corresfolder
savefile_prefix = 'ciCOH_' + helmes_show + eventphase + '_' + strfreq + '_lowweight' + str(lowweight)

## extract Channel Information

In [13]:
### load channel information ###
df = pd.read_csv(chnInf_file, header = 0)


chnInf_brainarea = df['brainarea'].to_numpy()

x, y = df['simulated_x'].to_numpy(), df['simulated_y'].to_numpy()
chnInf_coord = np.concatenate((np.expand_dims(x, axis = 1), np.expand_dims(y, axis = 1)), axis = 1)

## extract ciCOH

In [14]:
# load dictionary ciCOH
dic_ciCOH = pickle.load(open(ciCOHfile, "rb" ))

# ciCOH_normal, ciCOH_mild: np.ndarray, shape = (nchns, nchns, ntrials)
ciCOH_normal, ciCOH_mild = dic_ciCOH['ciCOH_normal'], dic_ciCOH['ciCOH_mild']

## ciCOH Visual

In [15]:
def ciCOH_visual_save(ciCOH, chn_area, chn_coord, str_recchnishow, lowweight, tobesavedfile, texts):
    """
        
        Args:
            
            ciCOH (np.sdarray): ciCOH Tensor for each trial (nchns, nchns)
            
            chn_area (list): a list of string representing brain area for each channel, len(chn_area) = nchns
            
            chn_coord (numpy.ndarray): x, y coordinates for each channel, shape = (nchns, 2)
            
            chn_recchni(list): showed recording chni for each channel, shape = (nchns, )
            
            lowweight: the threshold lowweight, only weight>lowweight is treated as connection
            
            tobesavedfile: file to save the visualized figure
            
            texts:
            
        Output:
            the visualizaton of ciCOH is saved in tobesavedfile
            
    """
    
    
    weight = abs(ciCOH)
    
    # change nan to 0
    ijs_nan = np.where(np.isnan(weight))

    for k in range(len(ijs_nan[0])):
        i_nan, j_nan = ijs_nan[0][k], ijs_nan[1][k]
        weight[i_nan][j_nan] = 0

    # weight > lowweight
    weight[np.where(weight<lowweight)] =0
    
    graph = graphVisual.graph_create(weight, vsarea = chn_area, vscoord = chn_coord, vschni = str_recchnishow)

    # set graph visualization style
    visual_style = graphVisual.graph_style(graph)
    

    igplot = graphVisual.graph_plot(graph, visual_style, texts = texts)
    
    igplot.save(tobesavedfile)
    
    return igplot

### remove data in the left hemesphere and Cd

In [16]:
# idx_left
idx_left = []

In [17]:
## remove the data in the left hemesphere 
if tag_showleft == False:

    for idx, area in enumerate(chnInf_brainarea.tolist()):
        if area[0] == 'l':
            idx_left.append(idx)

In [18]:
#  remove the chnsarea row of the left hemesphere    
chn_area = np.delete(chnInf_brainarea, idx_left, axis =0).tolist()

#  remove the chnInf_coord row of the left hemesphere       
chn_coord = np.delete(chnInf_coord, idx_left, axis = 0)

### normal

In [31]:
"""
    normal condition
"""

ciCOH = ciCOH_normal
condition = 'normal'

# remove the data in the left hemesphere
ciCOH = np.delete(ciCOH, idx_left, axis = 0)
ciCOH = np.delete(ciCOH, idx_left, axis = 1)


str_recchnishow = ['']*ciCOH.shape[0]

# save file config
savefilename = savefile_prefix + '_' + condition + '.png'
savefile = os.path.join(savefolder, savefilename)

# printed text config
texts = texts_area.copy()
title = title_pre + condition.capitalize()  + ' - Freq [' + str(freq[0]) + ' ' + str(freq[1]) + ']'
texts[title] = pos_title



# visualize and save
igplot = ciCOH_visual_save(ciCOH, 
                           chn_area = chn_area, 
                           chn_coord = chn_coord,
                           str_recchnishow = str_recchnishow,
                           lowweight = lowweight,  
                           tobesavedfile = savefile, 
                           texts = texts)

igplot.show()

print("Figure saved to " + savefile)
del ciCOH, condition
del str_recchnishow, savefilename, savefile, texts, title, igplot

Figure saved to /home/lingling/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/pipeline/NHP_Pinky/1_dataAnaly/FCAnaly/SKT/m3_SKTData_Reach_VisualciCOH/ciCOH_left_rightreach_freq9_11_lowweight0.25_normal.png


### mild

In [32]:
"""
    mild condition
"""

ciCOH = ciCOH_mild
condition = 'mild'

# remove the data in the left hemesphere
ciCOH = np.delete(ciCOH, idx_left, axis = 0)
ciCOH = np.delete(ciCOH, idx_left, axis = 1)


str_recchnishow = ['']*ciCOH.shape[0]

# save file config
savefilename = savefile_prefix + '_' + condition + '.png'
savefile = os.path.join(savefolder, savefilename)

# printed text config
texts = texts_area.copy()
title = title_pre + condition.capitalize()  + ' - Freq [' + str(freq[0]) + ' ' + str(freq[1]) + ']'
texts[title] = pos_title



# visualize and save
igplot = ciCOH_visual_save(ciCOH, 
                           chn_area = chn_area, 
                           chn_coord = chn_coord,
                           str_recchnishow = str_recchnishow,
                           lowweight = lowweight,  
                           tobesavedfile = savefile, 
                           texts = texts)

igplot.show()

print("Figure saved to " + savefile)
del ciCOH, condition
del str_recchnishow, savefilename, savefile, texts, title, igplot

Figure saved to /home/lingling/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/pipeline/NHP_Pinky/1_dataAnaly/FCAnaly/SKT/m3_SKTData_Reach_VisualciCOH/ciCOH_left_rightreach_freq9_11_lowweight0.25_mild.png
